# Prepare input data

In [1]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../images/crosslight/train',  # this is the target directory
        target_size=(112, 112),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../images/crosslight/validation',
        target_size=(112, 112),
        batch_size=batch_size,
        class_mode='binary')

Using TensorFlow backend.


Found 128 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


# Create the model

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(112, 112, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 110, 110, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 110, 110, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 16)        2320      
_________________________________________________________________
activation_2 (Activation)    (None, 53, 53, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 16)        2320      
__________

# Train the network

In [4]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1600 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=16  // batch_size)

Epoch 1/10
100/100 [==============================] - 17s - loss: 0.7015 - acc: 0.4850 - val_loss: 0.6930 - val_acc: 0.5000

In [5]:
model.save_weights('crosslight.h5')  # always save your weights after training or during training

# Test the network

In [6]:
batch_size = 16
test_generator = test_datagen.flow_from_directory(
        '../images/crosslight/test',
        target_size=(112, 112),
        batch_size=batch_size,
        class_mode='binary')

Found 16 images belonging to 2 classes.


In [7]:
model.evaluate_generator(
        test_generator,
        steps=1)

[0.0013851710828021169, 1.0]